# Ассоциативные правила. Практика

Строить ассоциативные правила мы будем на основе датасета от Netflix.

Примечание: так как предобработка датасета занимает большое время, мы сделали её за вас! Готовый датасет можно найти здесь.

Также вам понадобятся данные с ID и названиями фильмов, которые можно найти здесь.

Если вам интересно самостоятельно обработать данные с самого начала, то можете скачать датасет, приложенный к уроку, и выполнить шаги, изложенные в скринкасте. Они также прописаны в приложенном к уроку ноутбуке.
https://www.kaggle.com/netflix-inc/netflix-prize-data

**Подготовить данные самому**

In [1]:
import pandas as pd

In [2]:
data_all = []
for i in range(4):
    data = pd.read_csv('/home/aprosvetov/netflix/combined_data_' + str(i+1)+'.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])
    data_all.append(data)

In [3]:
df = pd.concat(data_all)

In [4]:
import numpy as np
df.index = np.arange(0,len(df))
print('Full dataset shape: {}'.format(df.shape))
print('-Dataset examples-')
print(df.iloc[::5000000, :])

Full dataset shape: (100498277, 2)
-Dataset examples-
           Cust_Id  Rating
0               1:     NaN
5000000    2560324     4.0
10000000   2271935     2.0
15000000   1921803     2.0
20000000   1933327     3.0
25000000   1465002     3.0
30000000    961023     4.0
35000000   1372532     5.0
40000000    854274     5.0
45000000    116334     3.0
50000000    768483     3.0
55000000   1331144     5.0
60000000   1609324     2.0
65000000   1699240     3.0
70000000   1776418     4.0
75000000   1643826     5.0
80000000    932047     4.0
85000000   2292868     4.0
90000000    932191     4.0
95000000   1815101     3.0
100000000   872339     4.0


In [ ]:
df.shape

In [ ]:
import numpy as np
df_nan = pd.DataFrame(pd.isnull(df.Rating))
df_nan = df_nan[df_nan['Rating'] == True]
df_nan = df_nan.reset_index()

movie_np = []
movie_id = 1

for i,j in zip(df_nan['index'][1:],df_nan['index'][:-1]):
    # numpy approach
    temp = np.full((1,np.abs(i-j-1)), movie_id)
    movie_np = np.append(movie_np, temp)
    movie_id += 1

# Account for last record and corresponding length
# numpy approach
last_record = np.full((1,len(df) - df_nan.iloc[-1, 0] - 1),movie_id)
movie_np = np.append(movie_np, last_record)

In [ ]:
df = df[pd.notnull(df['Rating'])]

df['Movie_Id'] = movie_np.astype(int)
df['Cust_Id'] = df['Cust_Id'].astype(int)
print('-Dataset examples-')
print(df.sample(3))

In [ ]:
df.to_csv('/srv/aprosvetov/netflix/data_prep.csv', sep=';', index = None)

**Взять уже подготовленные данные**

Подготовленные данные в data_fin.csv, но 100 млн.строк юпитер не может прочитать, падает ядро. Почистил файл и оставил 30 млн. data_fin_half.csv

In [1]:
!ls -la ../data/ | grep data_fin

-rw-r--r--  1  501 dialout 1806696634 Aug 22 17:26 data_fin.csv
-rw-r--r--  1 root root     566312581 Aug 22 17:56 data_fin_half.csv


In [8]:
!pip install --user apyori

  Created wheel for apyori: filename=apyori-1.1.2-cp36-none-any.whl size=5975 sha256=fafd95a7196bc8ba9b5a2503e7284514d07d28a019ebdc574a012b4b29b6ad2c
  Stored in directory: /notebooks/home/.cache/pip/wheels/5d/92/bb/474bbadbc8c0062b9eb168f69982a0443263f8ab1711a8cad0
Successfully built apyori
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
import pandas as pd

Итак, для начала откроем наш (уже предобработанный) датасет:

In [4]:
df = pd.read_csv('../data/data_fin_half.csv', sep=';')
df.head()

,Cust_Id,Rating,Movie_Id
0,1488844,3.0,1
1,822109,5.0,1
2,885013,4.0,1
3,30878,4.0,1
4,823519,3.0,1


Получаем следующее:

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30480507 entries, 0 to 30480506
Data columns (total 3 columns):
Cust_Id     int64
Rating      float64
Movie_Id    int64
dtypes: float64(1), int64(2)
memory usage: 697.6 MB


Здесь есть ID для пользователя, ID для фильма и рейтинг, который поставил данный пользователь данному фильму.

Из полученного датасета возьмем только те записи, у которых наивысший рейтинг (5) и объединим их по "Cust_Id". Фильмы сгруппируем в строчку с разделителем "пробел" так, чтобы для каждого пользователя была строка с Id тех фильмов, которые ему понравились:

In [5]:
good = df[df['Rating']==5].groupby('Cust_Id')['Movie_Id'].apply(lambda r: ' '.join([str(A) for A in r]))

Получаем датасет следующего вида:

In [6]:
good.head()

Cust_Id
6     12918 13728 13883 14187 14240 14454 14482 1455...
7     12693 12778 12870 12904 13052 13072 13123 1329...
8     12732 13651 14149 14367 15124 15887 16242 1676...
10    13195 13748 13955 14050 14313 14367 14621 1472...
25                                          15107 15270
Name: Movie_Id, dtype: object

Сначала идёт ID пользователя, дальше через пробел фильмы, которые нравятся этому пользователю.

Для дальнейших операций загружаем библиотеку apyori, которая представляет из себя реализацию алгоритмов Apriori в языке Python. Эта библиотека не является предустановленной, поэтому первоначально установите ее (например, через менеджер pip)

In [9]:
import apyori

Теперь, когда необходимая библиотека подгружена, сделаем несколько ассоциативных правил. Мы можем регулировать их количество, меняя параметры алгоритмов. Посмотрим, какие ассоциативные правила получаются для support = 0.04

In [10]:
association_rules = apyori.apriori(good.apply(lambda r: r.split(' ')), 
                                   min_support=0.04, 
                                   min_confidence=0.1, min_lift=2, 
                                   min_length=2)


Напомним, что support, confidence и lift — это показатели, которые мы учились рассчитывать ранее. Мы можем регулировать их для того, чтобы настроить ассоциативные правила.

Строго говоря, мы получили не сами ассоциативные правила, а генератор. Это можно проверить, если, например, вызвать переменную association_rules:

In [11]:
association_rules

<generator object apriori at 0x7f960c14a360>

Пройдемся по генератору и объединим его результаты. Эта процедура может занять некоторое время.

In [12]:
asr_df = pd.DataFrame(columns = ['from', 'to', 'confidence', 'support', 'lift'])
for item in association_rules:
    pair = item[0] 
    items = [x for x in pair]
    asr_df.loc[len(asr_df), :] =  ' '.join(list(item[2][0][0])), \
                                  ' '.join(list(item[2][0][1])),\
                                  item[2][0][2], item[1], item[2][0][3]

Посмотрим, что получилось:

In [13]:
asr_df

,from,to,confidence,support,lift
0,12870,13728,0.345912,0.0486241,2.14654
1,12870,14240,0.454075,0.0638284,2.11231
2,12870,14550,0.595409,0.0836955,2.70152
3,12870,14621,0.325363,0.0457357,2.10945
4,12870,14691,0.336783,0.0473409,2.05946
...,...,...,...,...,...
90,14240,14961 16377,0.18832,0.0404823,4.17229
91,14240,14961 16954,0.256678,0.055177,4.20926
92,14240,14961 17157,0.201692,0.0433568,4.20284
93,14240,16265 16954,0.220548,0.0474104,3.27853


In [14]:
asr_df.sort_values('lift').tail(10)

,from,to,confidence,support,lift
56,14961,16265,0.490286,0.0656444,3.81504
25,14240,14961,0.540143,0.116112,4.03423
90,14240,14961 16377,0.18832,0.0404823,4.17229
80,14240,14961 14550,0.281741,0.0605647,4.18257
92,14240,14961 17157,0.201692,0.0433568,4.20284
91,14240,14961 16954,0.256678,0.055177,4.20926
86,14240,14961 14691,0.270395,0.0581256,4.23367
89,14240,16265 14961,0.278897,0.0599532,4.2486
85,14240,14961 14621,0.217596,0.0467757,4.26421
31,14302,16147,0.741323,0.0433406,11.8443


Мы видим здесь таблицу, где 2023 ассоциативных правила, и для каждого рассчитаны известные нам показатели.

Для того чтобы перейти от Id фильмов, к их названиям, нужно загрузить еще один файл, в котором содержится Id фильма, год его производства и название:

In [16]:
titles = pd.read_csv('../data/movie_titles.csv', encoding = "ISO-8859-1", 
                     header = None, 
                     names = ['Movie_Id', 'Year', 'Name'])

Мы можем написать процедуру, которая будет выводить названия фильмов в ассоциативном правиле и фильм, которое это ассоциативное правило рекомендует:

In [66]:
def get_rule_title(rule):
    display(rule)
    print(titles[titles.Movie_Id.isin(rule['from'].split(' '))]['Name'].values)
    print('----------->')
    print(titles[titles.Movie_Id.isin(rule['to'].split(' '))]['Name'].values)

Можем посмотреть, как выглядит это правило. Например, вызовем сотое правило:

In [33]:
get_rule_title(asr_df.loc[45])

['The Shawshank Redemption: Special Edition']
----------->
['Saving Private Ryan']


То есть, если человеку нравится фильм «Монстры», то мы советуем ему посмотреть фильм «В поисках Немо».

Перейдём к построению рекомендаций для случайного человека под id=159992. Посмотрим, какие фильмы он смотрел и как он их оценил. 

In [89]:
j = 14482

titles[titles.Movie_Id.isin(good.iloc[j].split(' '))]['Name']

12833                         Family Guy: Vol. 2: Season 3
13980                                  The Virgin Suicides
14549            The Shawshank Redemption: Special Edition
14620                                  Shrek (Full-screen)
14690                                           The Matrix
14724                Austin Powers: The Spy Who Shagged Me
14960    Lord of the Rings: The Return of the King: Ext...
15061                                               Grease
15843                                  Remember the Titans
15901                   Blue Collar Comedy Tour: The Movie
16301                      Family Guy: Vol. 1: Seasons 1-2
16376                                       The Green Mile
16437                                   The Wedding Singer
17298                                                 Clue
Name: Name, dtype: object

Как мы можем посчитать посчитать рекомендации для этого человека? Мы можем пройтись по всем правилам в нашей таблице и проверить, если они присутствуют в просмотрах человека и у них высокий рейтинг, значит это правило ему подходит и мы можем добавить этот фильм в список рекомендаций.

In [86]:
def print_rule_title(rule):
    
    return (titles[titles.Movie_Id.isin(rule['to'].split(' '))]['Name'].values)

In [90]:
result = []
for A in asr_df.index:
    if len(set(good.iloc[j].split(' ')) & set(asr_df['from'].loc[A].split(' '))) == len(asr_df['from'].loc[A].split(' ')):
        result.append(print_rule_title(asr_df.loc[A])[0])
print(set(result))

{'Lord of the Rings: The Return of the King: Extended Edition', 'Dead Poets Society', 'The Matrix', 'Star Wars: Episode IV: A New Hope', 'A Few Good Men', 'Indiana Jones and the Last Crusade', 'Shrek (Full-screen)', 'Saving Private Ryan', 'Titanic', 'The Green Mile', "Ocean's Eleven", 'The Fugitive', "Harry Potter and the Sorcerer's Stone"}


Если вы запустите этот код, то увидите, что здесь в рекомендации будут два фильма:

'Pretty Woman', 'Dirty Dancing'
Сложность такой рекомендательной системы состоит в том, что мы ограничены теми ассоциативными правилами, которые были созданы. Если фильм редкий, то он в эти ассоциативные правила, скорее всего, не попадёт.

Также для разных людей будет разное количество рекомендаций, в зависимости от того, сколько фильмов посмотрел каждый из них. И, если нам захочется порекомендовать каждому пользователю минимум три фильма, то не с каждым пользователем мы сможем это сделать.

Эту проблему можно решить с помощью алгоритма Коллаборативная фильтрация. 

# Задания

Задание 9.4.1

1. Найдите фильмы, которые понравились пользователю с ID, равным 130. Понравившимися пользователю фильмами мы будем считать те фильмы, которым он поставил наивысшую оценку (5). Скопируйте все ID фильмов. Например: 68 943 325 1234.

In [93]:
j = 130

def print_rule_title_id(rule):
        return (titles[titles.Movie_Id.isin(rule['to'].split(' '))]['Movie_Id'].values)

result = []
for A in asr_df.index:
    if len(set(good.iloc[j].split(' ')) & set(asr_df['from'].loc[A].split(' '))) == len(asr_df['from'].loc[A].split(' ')):
        result.append(print_rule_title_id(asr_df.loc[A])[0])
print(set(result))

{16377, 16954, 15124, 17157}


2. Найдите ассоциативное правило с индексом 315. Введите название рекомендуемого фильма без кавычек.

In [96]:
asr_df

,from,to,confidence,support,lift
0,12870,13728,0.345912,0.0486241,2.14654
1,12870,14240,0.454075,0.0638284,2.11231
2,12870,14550,0.595409,0.0836955,2.70152
3,12870,14621,0.325363,0.0457357,2.10945
4,12870,14691,0.336783,0.0473409,2.05946
...,...,...,...,...,...
90,14240,14961 16377,0.18832,0.0404823,4.17229
91,14240,14961 16954,0.256678,0.055177,4.20926
92,14240,14961 17157,0.201692,0.0433568,4.20284
93,14240,16265 16954,0.220548,0.0474104,3.27853


3. Постройте рекомендацию для пользователя с ID = 21. Найдите рекомендованный фильм с самым коротким названием. Введите его без кавычек.

In [97]:
j = 21

result = []
for A in asr_df.index:
    if len(set(good.iloc[j].split(' ')) & set(asr_df['from'].loc[A].split(' '))) == len(asr_df['from'].loc[A].split(' ')):
        result.append(print_rule_title(asr_df.loc[A])[0])
print(set(result))

set()
